# CPLEX Solver WIKI

Provide an array containing dictionaries that define your variables an constraints

## Variables defintion

This example show you how to define a decision variable:

```
Xs = {
    "name" : [X(i) for i in I],
    "coef" : [c[i] for i],
    "type" : ["C" if relaxation else "I" for i in I],
    "ub"   : [1 for i in I],
    "lb"   : [0 for i in I],
}
```

You can also use the compact syntax:

```
Xs = {
    "loop" : "for i in I"
    "name" : "X(i)",
    "coef" : "c[i]",
    "type" : ""C" if relaxation else "I"",
    "ub"   : "1",
    "lb"   : "0",
}
```

by adding the attribute loop the function will recreate the list by evaluating the composed of the attribute and the loop expression.

# Source code

In [25]:
%%file "./cplex_solve/from_template_extract_variables.py"

def from_template_extract_variables(variables):
    obj = [];ub = [];lb = [];colnames = [];types = [];
    for v in variables:
        if "loop" in v:
            ## evaluation of the loop
            v["coef"] = eval("[" + v["coef"] + v["loop"] + "]")
            v["ub"]   = eval("[" + v["ub"] + v["loop"] + "]")
            v["lb"]   = eval("[" + v["lb"] + v["loop"] + "]")
            v["name"] = eval("[" + v["name"] + v["name"] + "]")
            v["type"] = eval("[" + v["type"] + v["loop"] + "]")
            
        # Concatenation of variables attributes
        obj      = obj + v["coef"]
        ub       = ub  + v["ub"]
        lb       = lb  + v["lb"]
        colnames = colnames + v["name"]
        types    = types + v["type"]
        
    return obj, ub, lb, colnames, types

Overwriting ./cplex_solve/from_template_extract_variables.py


In [26]:
%%file "./cplex_solve/from_template_extract_constraints.py"

def from_template_extract_constraints(constraints):
    rows = []; senses = []; rhs = [];
    for c in constraints:
        if "loop" in c:
            c["lin_expr"] = eval("[" + c["lin_expr"] + c["loop"] + "]")
            c["senses"]   = eval("[" + c["senses"] + c["loop"] + "]")
            c["rhs"]      = eval("[" + c["rhs"] + c["loop"] + "]")
            
        # Concatenation of variables attributes    
        rows   = rows   + c["lin_expr"]
        senses = senses + c["senses"]
        rhs    = rhs    + c["rhs"]
        
    return rows, senses, rhs

Overwriting ./cplex_solve/from_template_extract_constraints.py


In [2]:
%%file "./cplex_solve/from_template.py"

import cplex
import cplex_solve

def from_template(variables,constraints,
                  minimize=True, path=None, verbose=False, solve=True):
    
    #####################################################################
    # Extract variables
    obj, ub, lb, colnames, types = cplex_solve.from_template_extract_variables(variables)
    #####################################################################
    # Extract constraints
    rows, senses, rhs = cplex_solve.from_template_extract_constraints(constraints)
    #####################################################################
    # Creating problem
    prob = cplex.Cplex()
    
    #Disable logging
    if not verbose:
        prob.set_log_stream(None)
        prob.set_error_stream(None)
        prob.set_warning_stream(None)
        prob.set_results_stream(None)
    
    ## Objective function sense
    if minimize:
        prob.objective.set_sense(prob.objective.sense.minimize)
    else:
        prob.objective.set_sense(prob.objective.sense.maximize)
    ## Objective function
    prob.variables.add(obj=obj,ub=ub,lb=lb,names=colnames,types=types)
    ## Constraintes
    prob.linear_constraints.add(lin_expr=rows,senses=senses, rhs=rhs)

    #####################################################################
    # Saving the linear problem formulation into a file
    if path:
        prob.write(path) # print the formulation into a file

    #####################################################################
    # Solving problem
    if solve:
        prob.solve()
    
    return prob

Overwriting ./cplex_solve/from_template.py


Solve a linear program with the form $$ \{ c x : A x \le b \} $$

In [17]:
%%file "./cplex_solve/from_matrices.py"

from cplex_solve.from_template import from_template

def from_matrices(c,A,B,relaxation=False,path=None,verbose=False,minimize=True,senses="L"):
    """
    c   coefficent of the objective function
    A   coefficent of the constaints
    B   Right-hand-side
    """
    #####################################################################
    # Decision variables
    
    def X(i):
        return "X_" + str(i)
    
    I = range(len(c)); J = range(len(B))
    #####################################################################
    # Objective function
    
    Xs = {
        "name" : [X(i) for i in I],
        "coef" : c,
        "type" : ["C" if relaxation else "I" for i in I],
        "ub"   : [],
        "lb"   : [0 for i in I],
    }

    variables = [Xs]
    #####################################################################
    # Constraints
    
    c1 = {
        "lin_expr": [[[X(i) for i in I], A[j]]
                     for j in J],
        "senses"  : [senses for j in J],
        "rhs"     : B
    }
    
    constraints = [c1]
    #####################################################################
    # Solving
    prob = from_template(variables,constraints,
                       minimize=minimize, path=path, verbose=verbose)

    return prob

Overwriting ./cplex_solve/from_matrices.py
